In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.Container {width:85% !important;}
div.CodeMirror {font-family:NanumGothic; font-size:13pt; line-height : 150%;}
div.output_area pre {font-family:NanumGothic; font-size:13pt; line-height : 150%;}
div.output_wrapper pre {font-family:NanumGothic; font-size:13pt; line-height : 150%;}
div.input {font-family:NanumGothic; font-size:13pt; line-height : 150%;}
</style>
"""))

# 각 포지션 별 세부 데이터 웹 크롤링

## 후스코어드에서 수비지표 가져오기

In [27]:
# ▶ 1. 필요한 패키지 로드
from selenium import webdriver # 동적 웹 크롤링
import time # 대기시간
import pandas as pd # 데이터 프레임화 해야하기 때문
import numpy as np
from pandas.io.html import read_html

In [52]:
# ▶ 2. 해당 리그 모든 선수의 수비 지표 크롤링
def defensive (url, sleep_time) : # url 주소와 대기시간을 매개변수로 받는다.
    chrome_driver = 'D:/Gray_Bigdata/src/07_Python/chromedriver.exe'
    driver = webdriver.Chrome(chrome_driver)
    driver.get(url)
    
    time.sleep(sleep_time)
    defense = driver.find_element_by_link_text('Defensive')
    defense.click()
    
    time.sleep(sleep_time)
    all_player = driver.find_element_by_link_text('All players')
    all_player.click()
    
    time.sleep(sleep_time)
    page = driver.find_element_by_link_text('last')
    total_page = int(page.get_attribute('data-page'))
    
    df_defensive = pd.DataFrame(columns = ['full_name', 'Apps', 'Mins', 'Tackles', 'Inter', 'Fouls',
       'Offsides', 'Clear', 'Drb', 'Blocks', 'OwnG', 'Rating'])
    
    for i in np.arange(total_page) :
        time.sleep(sleep_time)
        table = driver.find_element_by_xpath('//*[@id="statistics-table-defensive"]') 
        # 전체 태그 중 id속성의 "" 사이의 속성을 가진 모든태그 선택
        table_html = table.get_attribute('innerHTML')
        
        df2 = read_html(table_html)[0] # 0번 인덱스에 모든 값이 들어가있음
        df_defensive = pd.concat([df_defensive, df2], axis=0)
        
        driver.find_element_by_link_text('next').click() # 크롤링 후 다음페이지로 이동
     
    return(df_defensive)

url = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/7361/Stages/16368/PlayerStatistics/England-Premier-League-2018-2019'
df = defensive(url, 2)

In [53]:
df.head()

,full_name,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating,Player,Player.1
0,NaN,32(5),2926,0.4,0.4,0.3,-,0.1,0.3,-,-,7.81,"1Eden HazardChelsea, 30, M(CLR),FW","Eden Hazard Chelsea, 30, M(CLR),FW"
1,NaN,31(3),2777,0.9,0.5,1.2,-,0.1,0.5,0.1,-,7.61,"2Raheem SterlingMan City, 26, M(CLR),FW","Raheem Sterling Man City, 26, M(CLR),FW"
2,NaN,37(1),3262,0.6,0.1,0.7,-,0.1,0.3,-,-,7.57,"3Mohamed SalahLiverpool, 29, AM(CLR),FW","Mohamed Salah Liverpool, 29, AM(CLR),FW"
3,NaN,31(2),2480,0.5,0.3,0.6,-,0.1,0.1,-,-,7.53,"4Sergio AgüeroMan City, 33, AM(CL),FW","Sergio Agüero Man City, 33, AM(CL),FW"
4,NaN,38,3385,1.0,1.1,0.3,1.1,5.2,-,0.5,-,7.44,"5Virgil van DijkLiverpool, 30, D(C)","Virgil van Dijk Liverpool, 30, D(C)"


In [54]:
# ▶ 3. 데이터 수집을 위한 데이터 전처리
# 쓸모없는 열 삭제
df = df.reset_index()
df.drop(['index', 'Player.1'] , axis=1, inplace=True)

df.head()

,full_name,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating,Player
0,NaN,32(5),2926,0.4,0.4,0.3,-,0.1,0.3,-,-,7.81,"1Eden HazardChelsea, 30, M(CLR),FW"
1,NaN,31(3),2777,0.9,0.5,1.2,-,0.1,0.5,0.1,-,7.61,"2Raheem SterlingMan City, 26, M(CLR),FW"
2,NaN,37(1),3262,0.6,0.1,0.7,-,0.1,0.3,-,-,7.57,"3Mohamed SalahLiverpool, 29, AM(CLR),FW"
3,NaN,31(2),2480,0.5,0.3,0.6,-,0.1,0.1,-,-,7.53,"4Sergio AgüeroMan City, 33, AM(CL),FW"
4,NaN,38,3385,1.0,1.1,0.3,1.1,5.2,-,0.5,-,7.44,"5Virgil van DijkLiverpool, 30, D(C)"


In [130]:
df2 = df.copy()

df2.head()

,full_name,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating,Player
0,NaN,32(5),2926,0.4,0.4,0.3,-,0.1,0.3,-,-,7.81,"1Eden HazardChelsea, 30, M(CLR),FW"
1,NaN,31(3),2777,0.9,0.5,1.2,-,0.1,0.5,0.1,-,7.61,"2Raheem SterlingMan City, 26, M(CLR),FW"
2,NaN,37(1),3262,0.6,0.1,0.7,-,0.1,0.3,-,-,7.57,"3Mohamed SalahLiverpool, 29, AM(CLR),FW"
3,NaN,31(2),2480,0.5,0.3,0.6,-,0.1,0.1,-,-,7.53,"4Sergio AgüeroMan City, 33, AM(CL),FW"
4,NaN,38,3385,1.0,1.1,0.3,1.1,5.2,-,0.5,-,7.44,"5Virgil van DijkLiverpool, 30, D(C)"


In [131]:
df2['full_name'] = df2['Player'].copy()

df2.head()

,full_name,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating,Player
0,"1Eden HazardChelsea, 30, M(CLR),FW",32(5),2926,0.4,0.4,0.3,-,0.1,0.3,-,-,7.81,"1Eden HazardChelsea, 30, M(CLR),FW"
1,"2Raheem SterlingMan City, 26, M(CLR),FW",31(3),2777,0.9,0.5,1.2,-,0.1,0.5,0.1,-,7.61,"2Raheem SterlingMan City, 26, M(CLR),FW"
2,"3Mohamed SalahLiverpool, 29, AM(CLR),FW",37(1),3262,0.6,0.1,0.7,-,0.1,0.3,-,-,7.57,"3Mohamed SalahLiverpool, 29, AM(CLR),FW"
3,"4Sergio AgüeroMan City, 33, AM(CL),FW",31(2),2480,0.5,0.3,0.6,-,0.1,0.1,-,-,7.53,"4Sergio AgüeroMan City, 33, AM(CL),FW"
4,"5Virgil van DijkLiverpool, 30, D(C)",38,3385,1.0,1.1,0.3,1.1,5.2,-,0.5,-,7.44,"5Virgil van DijkLiverpool, 30, D(C)"


In [132]:
df2.drop(['Player'] , axis=1, inplace=True)

df2.head()

,full_name,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating
0,"1Eden HazardChelsea, 30, M(CLR),FW",32(5),2926,0.4,0.4,0.3,-,0.1,0.3,-,-,7.81
1,"2Raheem SterlingMan City, 26, M(CLR),FW",31(3),2777,0.9,0.5,1.2,-,0.1,0.5,0.1,-,7.61
2,"3Mohamed SalahLiverpool, 29, AM(CLR),FW",37(1),3262,0.6,0.1,0.7,-,0.1,0.3,-,-,7.57
3,"4Sergio AgüeroMan City, 33, AM(CL),FW",31(2),2480,0.5,0.3,0.6,-,0.1,0.1,-,-,7.53
4,"5Virgil van DijkLiverpool, 30, D(C)",38,3385,1.0,1.1,0.3,1.1,5.2,-,0.5,-,7.44


In [133]:
# Player에 이름만 넣기
spl = df2['full_name'].str.split(',')

df2.head()

,full_name,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating
0,"1Eden HazardChelsea, 30, M(CLR),FW",32(5),2926,0.4,0.4,0.3,-,0.1,0.3,-,-,7.81
1,"2Raheem SterlingMan City, 26, M(CLR),FW",31(3),2777,0.9,0.5,1.2,-,0.1,0.5,0.1,-,7.61
2,"3Mohamed SalahLiverpool, 29, AM(CLR),FW",37(1),3262,0.6,0.1,0.7,-,0.1,0.3,-,-,7.57
3,"4Sergio AgüeroMan City, 33, AM(CL),FW",31(2),2480,0.5,0.3,0.6,-,0.1,0.1,-,-,7.53
4,"5Virgil van DijkLiverpool, 30, D(C)",38,3385,1.0,1.1,0.3,1.1,5.2,-,0.5,-,7.44


In [137]:
player = []
for i in range(len(spl)):
    a = spl[i][0]
    player.append(a)

df2['full_name'] = player

In [138]:
df2.head()

,full_name,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating
0,1Eden HazardChelsea,32(5),2926,0.4,0.4,0.3,-,0.1,0.3,-,-,7.81
1,2Raheem SterlingMan City,31(3),2777,0.9,0.5,1.2,-,0.1,0.5,0.1,-,7.61
2,3Mohamed SalahLiverpool,37(1),3262,0.6,0.1,0.7,-,0.1,0.3,-,-,7.57
3,4Sergio AgüeroMan City,31(2),2480,0.5,0.3,0.6,-,0.1,0.1,-,-,7.53
4,5Virgil van DijkLiverpool,38,3385,1.0,1.1,0.3,1.1,5.2,-,0.5,-,7.44


In [ ]:
# '-'로 처리된 부분 NaN으로 처리하기

In [ ]:
# csv파일로 저장하기

## 후스코어드에서 미드필드 지표 가져오기

## 후스코어드에서 공격지표 가져오기